In [105]:
#Selenium imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

In [72]:
#Other imports here
import os
import wget
import time
import json
import copy
import random
import datetime
import os
import itertools

In [73]:
class Post:
    def __init__(self, href):
        self.href = href
        self.json_name = RedditStructure.get_json_name_from_href(href)
        self.comments = []
        self.retrieved_at = None
        
    def __str__(self):        
        return "(" + str(self.href) + ", " + str(self.json_name) + ", " + str(self.comments) + ", " + str(self.retrieved_at) + ")"

    def to_json(self):        
        obj_dict = copy.deepcopy(self.__dict__)
        
        if self.comments:
            obj_dict['comments'] = list(self.comments)
        
        with open(self.json_name + ".json", "w", encoding="utf-8") as out:
            out.write(json.dumps(obj_dict))
    

In [155]:
class RedditStructure:

    username = (By.CSS_SELECTOR, "input[name='username']")
    password = (By.CSS_SELECTOR, "input[name='password']") 
    submit = (By.XPATH, "//*[contains(text(), 'Log In')]")
    # save_information = (By.XPATH, "//div[contains(text(), 'Agora não') or contains(text(), 'Not now')]") 
    # not_now_notification = (By.XPATH, "//button[contains(text(), 'Agora não') or contains(text(), 'Not now')]") 
    
    # JScript to scroll a scroll bar
    js_script_to_scroll = """ 
        arguments[0].scrollTo(0, arguments[0].scrollHeight);
        return arguments[0].scrollHeight; """
    
    # --------------------------------------------------------------------
    
    @staticmethod
    def __get_random_sleep__(minimum, maximum):        
        time.sleep(random.uniform(minimum, maximum))
    
    
    @staticmethod
    def sleep(time):        
        RedditStructure.__get_random_sleep__(time-2, time+2)


    @staticmethod
    def get_json_name_from_href(href):
        _, _, _, _, subreddit, _, query = href.split('/')
        query = query.split('&')[0]
        return str(subreddit) + '-' + str(query)
    

    @staticmethod
    def get_json_name_from_post(identifier):
        identifier = identifier.split('/')
        _, _, subreddit, _, code, title, _ = identifier
        return str(subreddit) + '-' + str(code) + '-' + str(tittle) 


    @staticmethod
    def get_login_clickable(driver):
        xpath = "//span[contains(text(), 'Log In')]"
        return driver.find_element(By.XPATH, xpath)
    
    
    @staticmethod
    def get_search_input(driver):
        xpath = "//span[@class='input-container activated']"
        return driver.find_element(By.XPATH, xpath)
        # return driver.find_element_by_xpath(xpath)
        

    @staticmethod
    def get_communities_clickable(driver):
        return driver.find_element(By.XPATH, "//span[contains(text(), 'Communities')]")


    @staticmethod
    def get_subreddit_clickable(driver, subreddit):
        return driver.find_element(By.XPATH, f"//a[contains(text(), '{subreddit}')]")
        
    
    @staticmethod
    def is_valid_handle(driver, user):
        try:
            InstaStructure.get_n_posts(driver)
            return True
        except:
            return False
    
    
    @staticmethod
    def is_public(driver, user):           
        xpath = f"//*[contains(text(), 'Esta conta é privada') or contains(text(), 'This account is private')]"
        try:
            private_element = driver.find_element_by_xpath(xpath)
            return False
        except:
            return True
            
     
    @staticmethod
    def get_n_posts(driver):       
        return driver.find_element_by_xpath("//*[contains(text(), 'publicaç') or contains(text(), 'post')]//*[not(contains(text(), 'publicaç')) and not(contains(text(), 'post'))]").text
    
    
    @staticmethod
    def get_followers_following_clickable(driver, which):        
        if which == 'followers':
            return driver.find_element_by_xpath("//*[contains(text(), 'seguidor') or contains(text(), 'follower')]")
        elif which == 'following':
            return driver.find_element_by_xpath("//*[contains(text(), 'seguindo') or contains(text(), 'following')]")
        else:
            raise Exception("Invalid clickable: followers or following")
    
    
    @staticmethod
    def get_followers(driver):       
        return driver.find_element_by_xpath("//*[contains(text(), 'seguidor') or contains(text(), 'follower')]//*[not(contains(text(), 'seguidor')) and not(contains(text(), 'follower'))]").text
    
    
    @staticmethod
    def get_following(driver):       
        return driver.find_element_by_xpath("//*[contains(text(), 'seguindo') or contains(text(), 'following')]//*[not(contains(text(), 'seguindo')) and not(contains(text(), 'following'))]").text
    
    
    @staticmethod
    def get_name(driver):       
        return driver.find_element_by_xpath("//*[@class='_aa_c']/*[1]").text
    
    
    @staticmethod
    def get_bio(driver):        
        return driver.find_element_by_xpath("//*[@class='_aa_c']/*[last()]").text
    
    
    @staticmethod
    def get_scroll_bar(driver, which):       
        if which == 'followers':
            return driver.find_element_by_xpath('//div[@class="_aano"]')
        elif which == 'following':
            return driver.find_element_by_xpath('//div[@class="_aano"]')
        else:
            raise Exception("Invalid clickable: followers or following")
        
    
    @staticmethod
    def get_links(scroll_bar):        
        return scroll_bar.find_elements_by_tag_name('a')
    
    @staticmethod
    def get_visible_posts(driver):       
        return driver.find_elements_by_xpath("//*[@class='_aabd _aa8k  _al3l']")
    
    
    @staticmethod
    def get_post_href(post):       
        return post.find_element_by_tag_name('a').get_attribute('href')
    
    
    @staticmethod
    def __get_caption_box__(driver):       
        try:
            caption_box = driver.find_element_by_xpath("//*[@class='_a9zr']")
        except:
            caption_box = None

        return caption_box
            
    
    @staticmethod
    def get_caption(driver):       
        caption_box = InstaStructure.__get_caption_box__(driver)
        
        try:
            caption = caption_box.find_element_by_tag_name('h1').text
        except:
            caption = None

        return caption
    
    
    @staticmethod
    def get_datetime(driver):        
        return driver.find_element_by_xpath("//time[@class='_aaqe']").get_attribute('datetime')
        

In [151]:
class RedditDriver:
    
    def __init__(self, username, password):        
        self.username = username
        self.password = password
        self.driver_directory = None
        self.driver = None
        self.mounted = False
        self.logged_in = False
        
        
    def mount_driver(self, driver_directory=None, overwrite=False):        
        # If directory not passed or there is already an opened driver and overwrite=False
        if driver_directory is None or (overwrite is False and self.driver is not None):
            raise Exception("Error found while mounting driver")
        
        # If should overwrite and there is an opened driver, quit it before opening new driver
        if overwrite is True and self.driver is not None:
            self.quit()
        
        self.driver_directory = driver_directory
        self.options = webdriver.ChromeOptions()
        self.options.add_argument("--start-fullscreen")

        self.driver = webdriver.Chrome(options=self.options)
        self.mounted = True
        
        
    def login(self, timeout=5):       
        if self.driver is None:
            raise Exception("Driver not mounted")
            
        if timeout < 0:
            raise Exception("Invalid timeout")
        
        self.driver.get("https://www.reddit.com/")

        RedditStructure.sleep(3)
        RedditStructure.get_login_clickable(self.driver).click()
        RedditStructure.sleep(3)
        
        username = WebDriverWait(self.driver, timeout).until(EC.element_to_be_clickable(RedditStructure.username))
        password = WebDriverWait(self.driver, timeout).until(EC.element_to_be_clickable(RedditStructure.password))
        username.clear()
        password.clear()
        username.send_keys(self.username)
        password.send_keys(self.password)
        password.send_keys(Keys.ENTER)
        
        # WebDriverWait(self.driver, timeout).until(EC.element_to_be_clickable(RedditStructure.submit)).click()
        RedditStructure.sleep(3)

        self.logged_in = True
        

    def quit(self):        
        self.driver.quit()
        self.driver = None
        self.mounted = False
        self.logged_in = False
        
        

In [143]:
class RedditBot:
    
    def __init__(self, insta_driver):
        
        self.insta_driver = insta_driver
        self.driver = self.insta_driver.driver
        
    
    def __get_list__(self, user=None, which='followers', sleep=5):   
        if which != 'followers' and which != 'following':
            raise Exception(f"Invalid list = {which}: choose either 'followers' or 'following'")
        
        # get the clickable scroll bar element and click it to open the followers/following page
        InstaStructure.get_followers_following_clickable(self.driver, which=which).click()
        InstaStructure.sleep(sleep)
        
        scroll_bar = InstaStructure.get_scroll_bar(self.driver, which)
        # height variable
        last_ht, ht = 0, 1
        
        # scroll the scroll bar until it reaches the bottom
        while last_ht != ht:
            last_ht = ht
            InstaStructure.sleep(sleep)
            ht = self.driver.execute_script(InstaStructure.js_script_to_scroll, scroll_bar)
    
        # once at the bottom, scrapes all links and names of followers/following
        links = InstaStructure.get_links(scroll_bar)
        names = [name.text for name in links if name.text != '']
        
        # return to the users page
        self.driver.back()
        
        return names
    
               
    def __get_posts__(self, user=None, pause=5):       
        # get driver internet page last height
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        user.posts = {}
        last_post_fetched = None
        
        while True:
            
            # filter list (takes out all posts already fetched)
            if last_post_fetched is None:
                # get all visible posts (given driver)            
                visible_posts = InstaStructure.get_visible_posts(self.driver)
            else:
                last_post_fetched = visible_posts[-1]
                
                visible_posts = InstaStructure.get_visible_posts(self.driver)
                
                # remove first until first is not the last post fetched
                while visible_posts[0] != last_post_fetched:
                    visible_posts = visible_posts[1:]
                    
                # only last post fetched remains in the list so remove it
                visible_posts = visible_posts[1:]
                
            
            for post in visible_posts:
                
                # get post href (/p/href)
                post_href = InstaStructure.get_post_href(post)
                
                # creates a post object with only the current post href
                current_post = Post(post_href)
                
                # if post not already fetched, fetch it
                if post_href not in user.posts:
                
                    # open the post (post here is an page element)
                    post.click()
                    
                    # inserts attributes for the current post
                    self.__get_post_info__(current_post)
                    
                    # get back to user post grid
                    self.driver.back()

                    # adds the post to the users post dict
                    user.posts.update({post_href: current_post})
            
            # scroll the page to make more posts visible
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            InstaStructure.sleep(pause)

            new_height = self.driver.execute_script("return document.body.scrollHeight")
            
            # will scroll the page until it can (until a new height is found)
            if new_height == last_height:
                break
                
            last_height = new_height
 

    def __get_post_info__(self, post=None, sleep=5):       
        if post is None or not isinstance(post, Post):
            raise Exception("Invalid post")
            
        InstaStructure.sleep(sleep)
        
        post.caption = InstaStructure.get_caption(self.driver)
        post.datetime = InstaStructure.get_datetime(self.driver)
    
    
    def get_users_info(self, user=None, sleep=5, get_followers=True, get_following=True, get_posts=True):       
        if user is None or not isinstance(user, User):
            raise Exception("Invalid user")
        
        self.driver.get(user.url)
        InstaStructure.sleep(sleep)
        
        if not InstaStructure.is_valid_handle(self.driver, user):
            raise Exception("Invalid user: handle invalid")
            
        if not InstaStructure.is_public(self.driver, user):
            raise Exception("Invalid user: private user")
                
        user.n_posts = int(InstaStructure.get_n_posts(self.driver).replace('.',''))
        user.followers = int(InstaStructure.get_followers(self.driver).replace('.',''))
        user.following = int(InstaStructure.get_following(self.driver).replace('.',''))
        user.name = InstaStructure.get_name(self.driver)
        user.bio = InstaStructure.get_bio(self.driver)
        
        if get_followers:
            user.followers_list = self.__get_list__(user)
            
        if get_following:
            user.following_list = self.__get_list__(user, which='following')
        
        if get_posts:
            self.__get_posts__(user=user)


    def get_into_subreddit(self, subreddit):
        search_bar = RedditStructure.get_search_input(self.driver)
        search_bar.send_keys(subreddit)
        search_bar.send_keys(Keys.ENTER)
        RedditStructure.get_communities_clickable(self.driver).click()
        RedditStructure.sleep(3)
        RedditStructure.get_subreddit_clickable(self.driver, subreddit).click()
        RedditStructure.sleep(3)
    
            
    def batch_process(self, subreddits, searches, posts_limit, comments_limit):
        log_name = datetime.datetime.now().strftime('%H_%M_%d_%m_%Y.log')
        
        with open(log_name, 'a', encoding="utf-8") as log:
            for subreddit in subreddits:
                try:
                    date_now = datetime.datetime.now().strftime('%H:%M of %m/%d/%Y')
                    self.get_into_subreddit(subreddit)
                    log.write(f"entered {subreddit} at {date_now}\n")
                except Exception as ex:
                    log.write(f"failed entering {subreddit} at {date_now} ({str(ex)})\n")
                
                for search in searches:
                    try:
                        date_now = datetime.datetime.now().strftime('%H:%M of %m/%d/%Y')
                        self.search_for(search)
                        log.write(f"searched for {search} at {date_now}\n")
                    except Exception as ex:
                        log.write(f"failed searching for {search} at {date_now} ({str(ex)})\n")


In [138]:
subreddits = ['r/brasilivre']
searches = []
# searches = ['preto', 'negro']
posts_limit = 5
comments_limit = 10

In [124]:
subreddits

['r/brasilivre']

In [158]:
driver = RedditDriver('pedrohenriquemarraarajo1@gmail.com', 'ajG7wkvT6$') # insert username and password
driver.mount_driver('chromedriver.exe')
driver.login()

In [159]:
bot = RedditBot(driver)

In [160]:
bot.batch_process(subreddits, searches, posts_limit, comments_limit)

In [ ]:
import pandas as pd
import os
import random
import json
import datetime

def create_dataframes(first_id=1000):
    jsons = list(filter(lambda file: file.endswith('.json'), os.listdir(os.getcwd())))
    random.shuffle(jsons)
    
    users = pd.DataFrame(columns=['user_id', 'bio', 'seguidos', 'seguidores', 'qtd_posts'])
    posts = pd.DataFrame(columns=['user_id', 'contador', 'legenda', 'polaridade', 'emocao', 'qtd_likes', 'horario'])
    
    for js_file in jsons:
        
        f = open(js_file)
        data = json.load(f)
        
        users.loc[len(users)] = [str(first_id), data['bio'], data['following'], data['followers'], data['n_posts']]
        
        data['posts'] = list(filter(lambda post: post['caption'] is not None, data['posts']))
        
        n = len(data['posts'])
        
        user_data = {
                        'user_id': [first_id for i in range(0, n)],
                        'contador': [i for i in range(0, n)],
                        'legenda': list(map(lambda post: post['caption'], data['posts'])),
                        'polaridade': ['' for i in range(0, n)],
                        'emocao': ['' for i in range(0, n)],
                        'qtd_likes': [0 for i in range(0, n)],
                        'horario': list(map(lambda post: datetime.datetime.strftime(datetime.datetime.strptime(post['datetime'], '%Y-%m-%dT%H:%M:%S.000Z'), '%Y-%m-%d %H:%M:%S'),
                                            data['posts']
                                           )
                                       )
                        }
        
        user_posts = pd.DataFrame(user_data)
        posts = pd.concat([posts, user_posts], ignore_index=True)
        
        first_id += 1
        
    return users, posts
        
    

In [ ]:
users, posts = create_dataframes()

In [ ]:
users

In [ ]:
posts

In [ ]:
users.to_csv('users_para_rotular.csv', index=False)

In [ ]:
posts.to_csv('para_rotular.csv', index=False)